<a href="https://colab.research.google.com/github/robbybrodie/time_as_computation_cost/blob/main/notebooks/03_Mode_Crowding_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mode Crowding Experiment

This notebook explores occupancy vs capacity with softmax distribution.

## Theory

- **K Modes**: Each with baseline utility u_i
- **Softmax Occupancy**: p_i(N) = softmax(u_i / τ) where τ = N
- **Crowding**: As N ↓, distribution crowds into top modes
- **Metrics**: Participation ratio, Gini coefficient, entropy

## Setup

In [ ]:
# Colab bootstrap
REPO_URL = "https://github.com/robbybrodie/time_as_computation_cost.git"
REPO_NAME = "time_as_computation_cost"

import pathlib
if not pathlib.Path(REPO_NAME).exists():
    !git clone $REPO_URL
%cd $REPO_NAME

if pathlib.Path("pyproject.toml").exists():
    !pip install -e .

In [ ]:
from experiments.run_mode_crowding import main
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import sys

# Ensure we can import the modules
repo_root = Path().resolve()
sys.path.insert(0, str(repo_root / "src"))

from tacc.crowding.mode_crowding import run_demo, run_experiment

## Basic Experiment

In [ ]:
# Run the basic experiment
results = main()

print("Mode crowding analysis completed!")
print(f"PR_min: {results['metrics']['PR_min']:.3f}")
print(f"Gini_max: {results['metrics']['Gini_max']:.3f}")
if results['metrics']['N_c_pr']:
    print(f"Critical N: {results['metrics']['N_c_pr']:.4f}")

## Interactive Demo

In [ ]:
# Create and display demo visualization
fig = run_demo(K=10, N_max=1.0, N_min=0.01, n_points=100)
plt.show()

## Parameter Exploration

In [ ]:
# Explore different numbers of modes
K_values = [5, 10, 20, 50]
results_by_K = []

for K in K_values:
    exp_result = run_experiment(K=K, n_points=50)
    results_by_K.append(exp_result)

# Plot scaling with K
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

PR_mins = [r['metrics']['PR_min'] for r in results_by_K]
Gini_maxs = [r['metrics']['Gini_max'] for r in results_by_K]

ax1.plot(K_values, PR_mins, 'bo-', linewidth=2, markersize=8)
ax1.set_xlabel('Number of Modes K')
ax1.set_ylabel('Minimum Participation Ratio')
ax1.set_title('PR_min vs K')
ax1.grid(True, alpha=0.3)

ax2.plot(K_values, Gini_maxs, 'ro-', linewidth=2, markersize=8)
ax2.set_xlabel('Number of Modes K')
ax2.set_ylabel('Maximum Gini Coefficient')
ax2.set_title('Gini_max vs K')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("K Scaling Summary:")
for i, K in enumerate(K_values):
    print(f"K={K}: PR_min={PR_mins[i]:.3f}, Gini_max={Gini_maxs[i]:.3f}")

## Utility Distribution Comparison

In [ ]:
# Compare random vs spaced utilities
utility_types = ['random', 'spaced']
K = 10

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

for i, utility_type in enumerate(utility_types):
    exp_result = run_experiment(K=K, utility_type=utility_type)
    
    # Plot utilities
    utilities = exp_result['utilities']
    if i == 0:
        ax1.bar(range(K), utilities, alpha=0.7, label=f'{utility_type.capitalize()}')
        ax1.set_xlabel('Mode Index')
        ax1.set_ylabel('Utility')
        ax1.set_title('Utility Distributions')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
    else:
        ax1.bar(range(K), utilities, alpha=0.7, label=f'{utility_type.capitalize()}')
        ax1.legend()
    
    # Plot crowding curves
    N_values = exp_result['curves']['N_values']
    PR_values = exp_result['curves']['participation_ratio']
    
    if i == 0:
        ax2.plot(N_values, PR_values, linewidth=2, label=f'{utility_type.capitalize()}')
    else:
        ax2.plot(N_values, PR_values, linewidth=2, label=f'{utility_type.capitalize()}')
    
    ax2.set_xlabel('Capacity N')
    ax2.set_ylabel('Participation Ratio')
    ax2.set_title('Participation Ratio vs N')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Plot Gini curves
    Gini_values = exp_result['curves']['gini_coefficient']
    
    if i == 0:
        ax3.plot(N_values, Gini_values, linewidth=2, label=f'{utility_type.capitalize()}')
    else:
        ax3.plot(N_values, Gini_values, linewidth=2, label=f'{utility_type.capitalize()}')
    
    ax3.set_xlabel('Capacity N')
    ax3.set_ylabel('Gini Coefficient')
    ax3.set_title('Gini Coefficient vs N')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Plot entropy curves
    entropy_values = exp_result['curves']['entropy']
    
    if i == 0:
        ax4.plot(N_values, entropy_values, linewidth=2, label=f'{utility_type.capitalize()}')
    else:
        ax4.plot(N_values, entropy_values, linewidth=2, label=f'{utility_type.capitalize()}')
    
    ax4.set_xlabel('Capacity N')
    ax4.set_ylabel('Shannon Entropy')
    ax4.set_title('Entropy vs N')
    ax4.legend()
    ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Critical Point Analysis

In [ ]:
# Detailed critical point analysis
thresholds = [0.05, 0.1, 0.2, 0.5]
K = 15

critical_points = []
for threshold in thresholds:
    exp_result = run_experiment(K=K, threshold=threshold, n_points=200)
    critical_points.append(exp_result['critical_analysis'])

# Plot derivative analysis for one threshold
detailed_result = run_experiment(K=K, threshold=0.1, n_points=200)
N_values = detailed_result['curves']['N_values']
critical_analysis = detailed_result['critical_analysis']

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Participation ratio and its derivative
PR_values = detailed_result['curves']['participation_ratio']
dPR_dN = critical_analysis['dPR_dN']

ax1.plot(N_values, PR_values, 'b-', linewidth=2, label='PR(N)')
if 'N_c_pr' in critical_analysis:
    ax1.axvline(x=critical_analysis['N_c_pr'], color='r', linestyle='--', 
                label=f'N_c = {critical_analysis["N_c_pr"]:.3f}')
ax1.set_xlabel('Capacity N')
ax1.set_ylabel('Participation Ratio')
ax1.set_title('Participation Ratio')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(N_values, dPR_dN, 'g-', linewidth=2, label='dPR/dN')
ax2.axhline(y=0.1, color='r', linestyle='--', alpha=0.7, label='Threshold = 0.1')
ax2.axhline(y=-0.1, color='r', linestyle='--', alpha=0.7)
ax2.set_xlabel('Capacity N')
ax2.set_ylabel('dPR/dN')
ax2.set_title('PR Derivative')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Gini coefficient and its derivative
Gini_values = detailed_result['curves']['gini_coefficient']
dGini_dN = critical_analysis['dGini_dN']

ax3.plot(N_values, Gini_values, 'r-', linewidth=2, label='Gini(N)')
if 'N_c_gini' in critical_analysis:
    ax3.axvline(x=critical_analysis['N_c_gini'], color='b', linestyle='--',
                label=f'N_c = {critical_analysis["N_c_gini"]:.3f}')
ax3.set_xlabel('Capacity N')
ax3.set_ylabel('Gini Coefficient')
ax3.set_title('Gini Coefficient')
ax3.legend()
ax3.grid(True, alpha=0.3)

ax4.plot(N_values, dGini_dN, 'm-', linewidth=2, label='dGini/dN')
ax4.axhline(y=0.1, color='r', linestyle='--', alpha=0.7, label='Threshold = 0.1')
ax4.axhline(y=-0.1, color='r', linestyle='--', alpha=0.7)
ax4.set_xlabel('Capacity N')
ax4.set_ylabel('dGini/dN')
ax4.set_title('Gini Derivative')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary of critical points vs threshold
print("Critical Point Detection vs Threshold:")
print("=" * 40)
for i, threshold in enumerate(thresholds):
    cp = critical_points[i]
    print(f"Threshold = {threshold}:")
    print(f"  PR critical: {cp['has_pr_critical']}")
    print(f"  Gini critical: {cp['has_gini_critical']}")
    print(f"  Entropy critical: {cp['has_entropy_critical']}")
    if cp['has_pr_critical']:
        print(f"  N_c (PR): {cp.get('N_c_pr', 'N/A')}")
    print()

## Key Insights

1. **Mode Crowding**: As capacity N decreases, occupancy crowds into highest utility modes
2. **Phase Transition**: Sharp transitions occur at critical capacity N_c
3. **Scaling**: More modes (larger K) → more dramatic crowding effects
4. **Utility Structure**: Spaced vs random utilities show different crowding patterns

## Physical Interpretation

- **Computational Modes**: Different "computational pathways" or "degrees of freedom"
- **Capacity Constraint**: Limited "computational resources" force selection
- **Critical Points**: Phase transitions in computational efficiency
- **Crowding**: Resource scarcity forces concentration into optimal modes

## Potential Applications

- **Information Processing**: Limited bandwidth forces mode selection
- **Quantum Systems**: Energy constraints limit accessible states
- **Neural Networks**: Capacity bottlenecks in deep learning

**Note**: This is a mathematical model requiring physical validation!